In [1]:
import os

os.chdir('/content/final-alpr-unconstrained-master/content/drive/MyDrive/Colab Notebooks/Nhan dien bien so Wpod-NET/final-alpr-unconstrained-master')
print(os.getcwd())

/content/final-alpr-unconstrained-master/content/drive/MyDrive/Colab Notebooks/Nhan dien bien so Wpod-NET/final-alpr-unconstrained-master


In [2]:
import sys
import numpy as np
import cv2
import argparse
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from random import choice
from os.path import isfile, isdir, basename, splitext
from os import makedirs

from src.keras_utils import save_model, load_model
from src.label import readShapes
from src.loss import loss
from src.utils import image_files_from_folder, show
from src.sampler import augment_sample, labels2output_map
from src.data_generator import DataGenerator

from pdb import set_trace as pause

In [3]:
def load_network(modelpath, input_dim):
    '''
    Hàm này load model và test luôn model xem nó có chạy đúng ko (có đúng là input shape > output shape 16 lần ko)
    '''
    model = load_model(modelpath)  # load eccv model theo mac dinh
    input_shape = (input_dim, input_dim, 3)  # cái này sẽ là (208,208,3)

    # Fixed input size for training
    inputs = keras.layers.Input(shape=(input_dim, input_dim, 3))
    outputs = model(inputs)

    output_shape = tuple([s for s in outputs.shape[1:]])
    output_dim = output_shape[1]
    model_stride = input_dim / output_dim

    assert input_dim % output_dim == 0, \
        'The output resolution must be divisible by the input resolution'

    assert model_stride == 2**4, \
        'Make sure your model generates a feature map with resolution ' \
        '16x smaller than the input'

    return model, model_stride, input_shape, output_shape


def process_data_item(data_item, dim, model_stride):
    # XX là ảnh đã augment
    # pts là 4 tọa độ của 4 đỉnh bounnding box đã chỉnh sửa theo ảnh augment. pts trong range(0,1)
    # llp là đối tượng Label được dựng lên từ pts thôi. thực ra label chính là pts mà
    # data_item[1].pts là dưới dạng tương đối
    XX, llp, pts = augment_sample(data_item[0], data_item[1].pts, dim) ## XX là (208,208,3)
    YY = labels2output_map(data_item[1].lp_type, llp, pts, dim, model_stride) ## YY là (13,13,10)  #### fix
    return XX, YY  # đây chính là cái mình dùng để train đây này.


def load_Data(data_dir):
    Files = image_files_from_folder(data_dir)  # Files là 1 list các img path

    Data = []
    for file in Files:
        # labfile là label file. File label của 1 ảnh có tên giống nó chỉ khác đuôi là txt
        labfile = splitext(file)[0] + '.txt'
        if isfile(labfile):
            L = readShapes(labfile)
            I = cv2.imread(file)
            Data.append([I, L[0]])  # I là ảnh, L[0] là label cho 4 góc của LP
            # L[0] thuộc class Shape()

    return Data


def process_Data(data, xshape=(208, 208, 3), yshape=(13, 13, 10)):
    X = np.empty((len(data),) + xshape)
    Y = np.empty((len(data),) + yshape)

    for (i, data_item) in enumerate(data):
        X[i], Y[i] = process_data_item(data_item, 208, 16)

    return X, Y


def plot_losses(train_loss, val_loss):
    plt.plot(train_loss)
    plt.plot(val_loss)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('iterations on batch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [4]:
# lấy giá trị các tham số từ command line
netname = 'my_trained_model'
train_dir = '/content/final-alpr-unconstrained-master/content/drive/MyDrive/Colab Notebooks/Nhan dien bien so Wpod-NET/final-alpr-unconstrained-master/samples/train_resized'
outdir = 'models/my-trained-model/'

iterations = 5000
batch_size = 64
dim = 208  # input_shape là (208,208,3)

In [5]:
if not isdir(outdir):
  makedirs(outdir)

In [ ]:
# create model
!python create-model.py eccv models/eccv-model-scracth

Creating model eccv
2022-01-11 17:59:40.490172: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Finished
Saving at models/eccv-model-scracth
/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [6]:
# file create_model.py tạo ra model rồi lưu vào 1 file
# hàm này load lại model đó vào đây
model, model_stride, xshape, yshape = load_network('models/eccv-model-scracth', dim) # xshape là (208,208,3), yshape là (13,13,10)
model.compile(loss=loss, optimizer=keras.optimizers.Adam(lr=0.001))  # compile model

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
print(xshape, yshape)

(208, 208, 3) (13, 13, 10)


In [8]:
print('Checking input directory...')
Data = load_Data(train_dir)
print('%d images with labels found' % len(Data))

Checking input directory...
206 images with labels found


In [9]:
dg = DataGenerator(	data=Data,
                    process_data_item_func=lambda x: process_data_item(x, dim, model_stride),
                    xshape=xshape,  # (208,208,3)
                    yshape=(yshape[0], yshape[1],yshape[2]),  # (13,13,10)
                    nthreads=2,
                    pool_size=1000,
                    min_nsamples=100)
dg.start()

In [10]:
Xtrain = np.empty((batch_size, dim, dim, 3), dtype='single')  # (batch,208,208,3)
Ytrain = np.empty((batch_size, int(dim/model_stride),int(dim/model_stride), 2*4+2))  ###### fix # (batch,13,13,10)

In [11]:
val_dir = './samples/val_resized'
Val_Data = load_Data(val_dir)
Xval, Yval = process_Data(Val_Data)  #### phải label cho cả tập val nữa

In [12]:
print(Xval.shape, Yval.shape)

(57, 208, 208, 3) (57, 13, 13, 10)


In [13]:
model_path_backup = '%s/%s_backup' % (outdir, netname)
model_path_final = '%s/%s_final' % (outdir, netname)

train_losses = []
val_losses = []

trainloss_txt = open('train_losses.txt', 'w')
valloss_txt = open('val_losses.txt', 'w')

In [14]:
for it in range(iterations):
  print('Iter. %d (of %d)' % (it+1, iterations))

  Xtrain, Ytrain = dg.get_batch(batch_size)
  train_loss = model.train_on_batch(Xtrain, Ytrain)
  val_loss = model.test_on_batch(Xval, Yval, reset_metrics=True, return_dict=False)

  print('\tLoss: %f' % train_loss)
  print('\tVal Loss: %f' % val_loss)

  train_losses.append(train_loss)
  val_losses.append(val_loss)
  trainloss_txt.write(str(train_loss)+'\n')
  valloss_txt.write(str(val_loss)+'\n')

  # Save model every 500 iterations
  if (it+1) % 500 == 0:
      print('Saving model (%s)' % model_path_backup)
      save_model(model, model_path_backup)

print('Stopping data generator')
dg.stop()

trainloss_txt.close()
valloss_txt.close()

print('Saving model (%s)' % model_path_final)
save_model(model, model_path_final)

Iter. 1 (of 5000)
	Loss: 569.789673
	Val Loss: 135.549713
Iter. 2 (of 5000)
	Loss: 276.996124
	Val Loss: 125.049515
Iter. 3 (of 5000)
	Loss: 137.753922
	Val Loss: 115.735680
Iter. 4 (of 5000)
	Loss: 103.639870
	Val Loss: 102.878845
Iter. 5 (of 5000)
	Loss: 101.438744
	Val Loss: 82.646950
Iter. 6 (of 5000)
	Loss: 86.974945
	Val Loss: 67.861061
Iter. 7 (of 5000)
	Loss: 74.487358
	Val Loss: 90.512650
Iter. 8 (of 5000)
	Loss: 63.931213
	Val Loss: 152.693192
Iter. 9 (of 5000)
	Loss: 60.225632
	Val Loss: 244.182877
Iter. 10 (of 5000)
	Loss: 71.122498
	Val Loss: 343.409882
Iter. 11 (of 5000)
	Loss: 82.766876
	Val Loss: 426.435852
Iter. 12 (of 5000)
	Loss: 68.564812
	Val Loss: 519.447449
Iter. 13 (of 5000)
	Loss: 47.814548
	Val Loss: 624.901550
Iter. 14 (of 5000)
	Loss: 67.554626
	Val Loss: 791.419495
Iter. 15 (of 5000)
	Loss: 50.253288
	Val Loss: 984.907349
Iter. 16 (of 5000)
	Loss: 59.563789
	Val Loss: 1144.574341
Iter. 17 (of 5000)
	Loss: 54.025940
	Val Loss: 1236.660522
Iter. 18 (of 5000)


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Streaming output truncated to the last 5000 lines.
	Loss: 3.730282
	Val Loss: 5.233164
Iter. 3337 (of 5000)
	Loss: 3.890761
	Val Loss: 5.121419
Iter. 3338 (of 5000)
	Loss: 4.278343
	Val Loss: 5.150769
Iter. 3339 (of 5000)
	Loss: 3.519644
	Val Loss: 5.122104
Iter. 3340 (of 5000)
	Loss: 5.459319
	Val Loss: 5.296442
Iter. 3341 (of 5000)
	Loss: 3.748498
	Val Loss: 5.302232
Iter. 3342 (of 5000)
	Loss: 3.969282
	Val Loss: 5.459352
Iter. 3343 (of 5000)
	Loss: 3.302789
	Val Loss: 6.020222
Iter. 3344 (of 5000)
	Loss: 3.853882
	Val Loss: 6.104669
Iter. 3345 (of 5000)
	Loss: 4.439738
	Val Loss: 5.990078
Iter. 3346 (of 5000)
	Loss: 4.282514
	Val Loss: 5.173689
Iter. 3347 (of 5000)
	Loss: 4.403347
	Val Loss: 4.868740
Iter. 3348 (of 5000)
	Loss: 4.517137
	Val Loss: 4.940539
Iter. 3349 (of 5000)
	Loss: 4.090705
	Val Loss: 4.992876
Iter. 3350 (of 5000)
	Loss: 3.702213
	Val Loss: 5.478343
Iter. 3351 (of 5000)
	Loss: 3.676352
	Val Loss: 6.297519
Iter. 3352 (of 5000)
	Loss: 4.556644
	Val Loss: 5.695805
I